In [3]:
import os
import sys
import datetime
from pathlib import Path
import logging
from main import SigmoidFitter, MeltomeAtlasHandler, setup_logging

import json
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed

now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M")

LOG_LEVEL = logging.DEBUG
LOG_FILE = f'./testing_{timestamp_str}.log'
setup_logging(LOG_FILE, LOG_LEVEL)

output_path = f'./results_meltome_{timestamp_str}'


2026-01-13 10:22:33 - root - INFO - Logging to file: ./testing_2026-01-13_10-22.log


In [4]:
# SCRIPT_DIR = Path(__file__).resolve().parent
CWD = Path(os.getcwd()).resolve()
PROJECT_ROOT = CWD.parent.parent
print(PROJECT_ROOT)
file_path = PROJECT_ROOT / "Data" / "FLIP" / "splits" / "meltome" / "full_dataset.json"
# print(file_path)

C:\Users\alexa\Documents\PROHITS


In [16]:
# Sample subset of Meltome
data_handler = MeltomeAtlasHandler(file_path, output_path, LOG_LEVEL)
print(data_handler.data.shape)
df = data_handler.select_subset(100)

# Meltome example
# example = data_handler.data.groupby(by='runName').sample(n=10, random_state=42)
# example.reset_index(inplace=True)
# example.drop(columns=['index'], inplace=True)
# example.to_json("./example/meltome_example.json", index=False, indent=4)
# chunk0 = data_handler.data.iloc[:25]
# data_handler.data = df 
# print(data_handler.data.shape)

2026-01-13 10:31:12 - MeltomeAtlasHandler - INFO - Meltome Handler initialization
2026-01-13 10:31:14 - MeltomeAtlasHandler - INFO - Meltome data loaded from C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json
2026-01-13 10:31:14 - MeltomeAtlasHandler - INFO - Meltome header ['proteinId', 'uniprotAccession', 'runName', 'meltingPoint', 'meltingBehaviour', 'sequence', 'quantNormMeltingPoint']
2026-01-13 10:31:14 - MeltomeAtlasHandler - INFO - Output directory: c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\results_meltome_2026-01-13_10-22
(221203, 7)


In [6]:
test = pd.read_json("./example/meltome_example.json")

In [38]:
for i, r in data_handler.data.iterrows():
    m = pd.DataFrame(r.meltingBehaviour)
    if len(m) > 10:
        print(i, m.shape)

2 (20, 3)
23 (20, 3)
24 (20, 3)
26 (20, 3)
28 (20, 3)
29 (20, 3)
35 (20, 3)
36 (20, 3)
37 (20, 3)
38 (20, 3)
39 (20, 3)
40 (20, 3)
41 (20, 3)
42 (20, 3)
43 (20, 3)
44 (20, 3)
45 (20, 3)
46 (20, 3)
47 (20, 3)
49 (20, 3)
50 (20, 3)
53 (20, 3)
54 (20, 3)
55 (20, 3)
56 (20, 3)
58 (20, 3)
60 (20, 3)
62 (20, 3)
63 (20, 3)
65 (20, 3)
70 (20, 3)
71 (20, 3)
72 (20, 3)
73 (20, 3)
74 (20, 3)
75 (20, 3)
76 (20, 3)
77 (20, 3)
79 (20, 3)
80 (20, 3)
81 (20, 3)
82 (20, 3)
83 (20, 3)
84 (20, 3)
85 (20, 3)
86 (20, 3)
87 (20, 3)
88 (20, 3)
89 (20, 3)
90 (20, 3)
91 (20, 3)
95 (20, 3)
96 (20, 3)
97 (20, 3)
98 (20, 3)
99 (20, 3)
100 (20, 3)
102 (20, 3)
103 (20, 3)
105 (20, 3)
107 (20, 3)
108 (20, 3)
109 (20, 3)
112 (20, 3)
116 (20, 3)
117 (20, 3)
119 (20, 3)
120 (20, 3)
121 (20, 3)
124 (20, 3)
125 (20, 3)
126 (20, 3)
127 (20, 3)
129 (20, 3)
130 (20, 3)
131 (20, 3)
132 (20, 3)
133 (20, 3)
134 (20, 3)
136 (20, 3)
137 (20, 3)
140 (20, 3)
141 (20, 3)
142 (20, 3)
143 (20, 3)
145 (20, 3)
146 (20, 3)
147 (20, 3)
1

KeyboardInterrupt: 

In [40]:
data_handler.data.iloc[37]

proteinId                                                 A0A087WPH5_Glb1l
uniprotAccession                                                A0A087WPH5
runName                                           Mus musculus BMDC lysate
meltingPoint                                                     50.151834
meltingBehaviour         [{'temperature': 37.0, 'channel': 'TMT126', 'f...
sequence                 XSLLLRLVVLLLLSQAEARSFVVDREHDRFLLDGVPFRYVSGSLHY...
quantNormMeltingPoint                                                  NaN
Name: 37, dtype: object

In [44]:
exp = pd.DataFrame(data_handler.data.meltingBehaviour.iloc[37])
exp[0::2]


,temperature,channel,fold_change
0,37.0,TMT126,0.980850
2,43.0,TMT127H,0.850087
4,39.9,TMT127L,1.244833
6,50.2,TMT128H,0.485396
8,46.6,TMT128L,0.747549
10,57.4,TMT129H,0.186827
12,53.8,TMT129L,0.488918
14,64.1,TMT130H,0.636655
16,61.0,TMT130L,0.264119
18,67.0,TMT131L,0.223476


In [45]:
exp[1::2]

,temperature,channel,fold_change
1,37.0,TMT126,0.995664
3,43.0,TMT127H,1.015515
5,39.9,TMT127L,0.970604
7,50.2,TMT128H,0.760739
9,46.6,TMT128L,0.767818
11,57.4,TMT129H,0.333589
13,53.8,TMT129L,0.476475
15,64.1,TMT130H,0.252042
17,61.0,TMT130L,0.306833
19,67.0,TMT131L,0.354270


In [ ]:
test

In [ ]:
data_handler.data = test

In [ ]:
os.path.exists(data_handler.output_dir)

In [ ]:
r = data_handler.process(10)

In [ ]:
pd.DataFrame(test.meltingBehaviour.loc[0])

In [ ]:
r = data_handler.process_parallel(10)

In [ ]:
test_data = pd.DataFrame(data_handler.data.iloc[0].meltingBehaviour)

In [ ]:
test_data

In [ ]:
t = SigmoidFitter()
print(t.get_statistics())
r = t.fit_curve(test_data.temperature.to_numpy(), test_data.fold_change.to_numpy())
print(t.get_statistics())

In [ ]:
x = t.get_melting_temp()
print(x)

In [ ]:
results0 = data_handler.process_chunk(chunk0)

In [ ]:
results0_df = pd.DataFrame(results0)

In [ ]:
data_handler.data

In [ ]:
results0_df

In [ ]:
results = data_handler.process(num_chunks=10)

In [ ]:
print(len(data_handler.data), len(results))

In [ ]:
pid, runName = results.loc[0][['pid', 'runName']]
print(pid, runName)
data_handler.data[(data_handler.data.uniprotAccession == pid) & (data_handler.data.runName == runName)]


In [ ]:
results[results.notna().all(axis=1)]

In [ ]:
results_na = results[results.isna().any(axis=1)]
print(results_na.shape)

In [ ]:
results_na